In [1]:
import fitz

In [2]:
doc = fitz.open("book.pdf")



In [9]:
text = ''
page = doc.load_page(23)
text = page.get_text()

In [10]:
text

'1955 and released for clinical use in 1960, other benzodiazepines—diazepam,\nlorazepam, and midazolam—came to be used extensively for premedication, conscious\nsedation, and induction of general anesthesia. Ketamine was synthesized in 1962 by\nStevens and first used clinically in 1965 by Corssen and Domino; it was released in\n1970 and continues to be popular today, particularly when administered in combination\nwith other agents for general anesthesia or when infused in low doses to awake patients\nwith painful conditions. Etomidate was synthesized in 1964 and released in 1972. Initial\nenthusiasm over its relative lack of circulatory and respiratory effects was tempered by\nevidence of adrenal suppression, reported after even a single dose. The release of\npropofol in 1986 (1989 in the United States) was a major advance in outpatient\nanesthesia because of its short duration of action (see Chapter 9). Propofol is currently\nthe most popular agent for intravenous induction worldwide.

In [12]:
from sentence_transformers import SentenceTransformer

/Volumes/Personal/anaconda3/envs/aba/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [22]:
model = SentenceTransformer("multi-qa-mpnet-base-dot-v1")
passages = text.split('\n\n') 
passage_embeddings = model.encode(passages, convert_to_numpy=  True)

In [24]:
import chromadb
from chromadb.config import Settings


client = chromadb.Client(Settings())

collection = client.create_collection(name="anaes_passages1")

# Add passages and their embeddings to the collection
for i, (passage, embedding) in enumerate(zip(passages, passage_embeddings)):
    collection.add(
        ids=[str(i)],
        metadatas=[{"text": passage}],
        embeddings=[embedding.tolist()]
    )



In [25]:
# Step 4: Retrieve Relevant Passages Using LangChain
def retrieve_relevant_passages(query, collection, model, top_k=5):
    query_embedding = model.encode([query], convert_to_numpy=True)
    results = collection.query(
        query_embeddings=[query_embedding.tolist()],
        n_results=top_k
    )
    return [result['metadata']['text'] for result in results['results'][0]['metadatas']]

In [26]:
query = "Generate multiple-choice questions about anesthesia techniques."
relevant_passages = retrieve_relevant_passages(query, collection, model)

# Step 5: Generate Questions Using BERT
question_generation_pipeline = pipeline("text2text-generation", model="t5-small")
for passage in relevant_passages:
    input_text = "Generate a question: " + passage
    question = question_generation_pipeline(input_text)
    print(question)

ValueError: Expected each value in the embedding to be a int or float, got an embedding with ['list'] - [[0.09474651515483856, -0.558135986328125, -0.2410891205072403, 0.0749274417757988, -0.44599634408950806, -0.20198345184326172, 0.4277171790599823, 0.28809434175491333, 0.08212965726852417, 0.4241153597831726, 0.10311483591794968, -0.08317455649375916, -0.2709055542945862, -0.10512807220220566, 0.3398776650428772, -0.05380271375179291, 0.005784943699836731, 0.15833021700382233, -0.2563202679157257, -0.1749650537967682, -0.31130632758140564, -0.2266230434179306, -0.45069390535354614, -0.32369256019592285, 0.1568351536989212, 0.21854785084724426, -0.23674452304840088, 0.017534498125314713, -0.10536240041255951, -0.34058892726898193, -0.288397878408432, 0.12683673202991486, -0.01950814761221409, -0.03590790182352066, -0.00010318074055248871, -0.4208744168281555, -0.35069042444229126, -0.05901409685611725, -0.4376811683177948, -0.13362552225589752, -0.05348239839076996, 0.244338124990463